# Deep Q-Learning para aterrizaje lunar

## Instalar paquetes y librerias

### Instalar Gymnasium

In [1]:
!pip install gymnasium
!pip install "gymnasium[atari, accept-rom-license]"
!apt-get install -y swig
!pip install gymnasium[box2d]

# gymnasium es una biblioteca utilizada para crear y gestionar entornos de simulación que se emplean principalmente en la investigación y desarrollo de algoritmos de aprendizaje por refuerzo (Reinforcement Learning, RL)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 37.0 MB/s eta 0:00:00
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=e4cb7cdf2b15ad8896286d22c584fb5991aea8a1fb17cadecb6bdbe5c6806240
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  swig4.0
Suggested packages:
  swig-doc swig-examples swig4.0-examples swig4.0-doc
The following NEW packages will be installed:
  

## Importar librerias

In [2]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

## Crear la estructura de la red neuronal

In [3]:
# Definir una clase llamada Network que hereda de nn.Module
class Network(nn.Module):

    # Definir el método constructor (__init__)
    def __init__(self, state_size, action_size, seed=42):
        # Llamar al constructor de la clase base (nn.Module)
        super(Network, self).__init__()

        # Fijar una semilla aleatoria para reproducibilidad
        self.seed = torch.manual_seed(seed)

        # Definir la primera capa completamente conectada (fc1)
        # Esta capa toma 'state_size' de entrada y produce 64 neuronas
        self.fc1 = nn.Linear(state_size, 64)

        # Definir la segunda capa completamente conectada (fc2)
        # Esta capa toma 64 neuronas de entrada y produce 64 neuronas de salida
        self.fc2 = nn.Linear(64, 64)

        # Definir la tercera capa completamente conectada (fc3)
        # Esta capa toma 64 neuronas de entrada y produce 'action_size' neuronas de salida
        self.fc3 = nn.Linear(64, action_size)

    # Definir el método de propagación hacia adelante (forward)
    # Este método define cómo los datos fluyen a través de la red
    def forward(self, state):
        # Pasar la entrada (state) a través de la primera capa (fc1)
        x = self.fc1(state)
        # Aplicar la función de activación ReLU a la salida de la primera capa
        x = F.relu(x)

        # Pasar la salida de la primera capa a la segunda capa (fc2)
        x = self.fc2(x)
        # Aplicar la función de activación ReLU a la salida de la segunda capa
        x = F.relu(x)

        # Pasar la salida de la segunda capa a la tercera capa (fc3)
        # La salida de esta capa es la salida final de la red
        return self.fc3(x)

## Entrenamiento

In [4]:
import gymnasium as gym
# Inicializa el entorno LunarLander-v2 utilizando la biblioteca gym
env = gym.make('LunarLander-v2')

# Obtiene la forma del espacio de observación (estado) del entorno
state_shape = env.observation_space.shape

# Obtiene el tamaño del estado, que es el número de elementos en el vector de estado
state_size = env.observation_space.shape[0]

# Obtiene el número de acciones posibles en el entorno
number_actions = env.action_space.n

print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

State shape:  (8,)
State size:  8
Number of actions:  4


## Inicializar hiperparámetros

In [5]:
learning_rate = 5e-4
minibatch_size = 100
discount_factor = 0.99
replay_buffer_size = int(1e5)
interpolation_parameter = 1e-3

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Implementar replay

In [6]:
class ReplayMemory(object):

  # Inicializa la clase con una capacidad fija para almacenar experiencias
  def __init__(self, capacity):
    # Configura el dispositivo de PyTorch: usa GPU si está disponible, de lo contrario usa CPU
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # Establece la capacidad máxima de la memoria de reproducción
    self.capacity = capacity
    # Inicializa una lista vacía para almacenar las experiencias (transiciones)
    self.memory = []

  # Método para agregar una nueva experiencia a la memoria
  def push(self, event):
    # Añade una nueva experiencia a la memoria
    self.memory.append(event)
    # Si la memoria excede su capacidad, elimina la experiencia más antigua (FIFO)
    if len(self.memory) > self.capacity:
      del self.memory[0]

  # Método para obtener una muestra aleatoria de experiencias de la memoria
  def sample(self, batch_size):
    # Selecciona una muestra aleatoria de experiencias de tamaño batch_size
    experiences = random.sample(self.memory, k=batch_size)
    # Extrae los estados de las experiencias y los convierte en un tensor de PyTorch
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    # Extrae las acciones y las convierte en un tensor de PyTorch
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    # Extrae las recompensas y las convierte en un tensor de PyTorch
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    # Extrae los próximos estados y los convierte en un tensor de PyTorch
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    # Extrae los valores de done (si el episodio ha terminado) y los convierte en un tensor de PyTorch
    dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
    # Retorna los diferentes componentes de la experiencia como tuplas de tensores
    return states, next_states, actions, rewards, dones


## Implementar DQN class

In [7]:
class Agent():

  # Inicializa el agente con los tamaños de estado y acción, y otros parámetros necesarios
  def __init__(self, state_size, action_size):
    # Configura el dispositivo de PyTorch: usa GPU si está disponible, de lo contrario usa CPU
    self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # Almacena el tamaño del estado (dimensiones del estado)
    self.state_size = state_size
    # Almacena el tamaño de las acciones (número de acciones posibles)
    self.action_size = action_size
    # Inicializa la red neuronal local (Q-network) que aprenderá a estimar valores Q
    self.local_qnetwork = Network(state_size, action_size).to(self.device)
    # Inicializa la red neuronal objetivo (target Q-network) utilizada para la estimación de valores objetivo
    self.target_qnetwork = Network(state_size, action_size).to(self.device)
    # Configura el optimizador Adam para actualizar los pesos de la red local
    self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
    # Inicializa la memoria de reproducción para almacenar experiencias
    self.memory = ReplayMemory(replay_buffer_size)
    # Inicializa un contador de pasos de tiempo
    self.t_step = 0

  # Método que toma un paso en el entorno, almacena la experiencia y aprende de manera intermitente
  def step(self, state, action, reward, next_state, done):
    # Almacena la experiencia (estado, acción, recompensa, siguiente estado, done) en la memoria
    self.memory.push((state, action, reward, next_state, done))
    # Incrementa el contador de pasos de tiempo y lo restablece cada 4 pasos
    self.t_step = (self.t_step + 1) % 4
    # Si es el momento de aprender y hay suficientes experiencias en la memoria, realiza el aprendizaje
    if self.t_step == 0:
      if len(self.memory.memory) > minibatch_size:
        # Muestra un lote de experiencias aleatorias y aprende de ellas
        experiences = self.memory.sample(100)
        self.learn(experiences, discount_factor)

  # Método para seleccionar una acción dada un estado, usando una política epsilon-greedy
  def act(self, state, epsilon=0.):
    # Convierte el estado en un tensor y lo pasa al dispositivo (CPU o GPU)
    state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
    # Coloca la red en modo evaluación (no se actualizarán los gradientes)
    self.local_qnetwork.eval()
    # Calcula los valores Q para el estado actual sin calcular los gradientes
    with torch.no_grad():
      action_values = self.local_qnetwork(state)
    # Regresa la red al modo de entrenamiento
    self.local_qnetwork.train()
    # Selecciona la acción con el valor Q más alto con probabilidad 1-epsilon, o una acción aleatoria con probabilidad epsilon
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  # Método que realiza el proceso de aprendizaje utilizando las experiencias de la memoria de reproducción
  def learn(self, experiences, discount_factor):
    # Extrae los estados, acciones, recompensas, próximos estados y dones de las experiencias
    states, next_states, actions, rewards, dones = experiences
    # Calcula los valores Q objetivo para el siguiente estado usando la red objetivo
    next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
    # Calcula los valores Q esperados sumando las recompensas y el valor descontado del próximo estado
    q_targets = rewards + discount_factor * next_q_targets * (1 - dones)
    # Calcula los valores Q estimados para las acciones tomadas utilizando la red local
    q_expected = self.local_qnetwork(states).gather(1, actions)
    # Calcula la pérdida (error) entre los valores Q esperados y los valores Q objetivo utilizando el error cuadrático medio
    loss = F.mse_loss(q_expected, q_targets)
    # Zera los gradientes acumulados
    self.optimizer.zero_grad()
    # Retropropaga la pérdida a través de la red
    loss.backward()
    # Actualiza los pesos de la red utilizando el optimizador Adam
    self.optimizer.step()
    # Realiza una actualización suave de la red objetivo para acercarla a la red local
    self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_parameter)

  # Método para realizar una actualización suave entre la red local y la red objetivo
  def soft_update(self, local_model, target_model, interpolation_parameter):
    # Actualiza los parámetros de la red objetivo con una combinación de los parámetros locales y los anteriores
    for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
      target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)


## Inicializar DQN agent

In [8]:
agent = Agent(state_size, number_actions)

## Entrenar DQN agent

In [9]:
number_episodes = 2000
maximum_number_timesteps_per_episode = 1000
epsilon_starting_value  = 1.0
epsilon_ending_value  = 0.01
epsilon_decay_value  = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen = 100)

# Bucle principal para entrenar al agente a través de múltiples episodios
for episode in range(1, number_episodes + 1):

  # Resetea el entorno al comienzo de cada episodio y obtiene el estado inicial
  state, _ = env.reset()

  # Inicializa la puntuación del episodio a cero
  score = 0

  # Bucle para cada paso de tiempo dentro de un episodio
  for t in range(maximum_number_timesteps_per_episode):

    # El agente elige una acción basada en el estado actual y la política epsilon-greedy
    action = agent.act(state, epsilon)

    # El entorno devuelve el siguiente estado, recompensa, y si el episodio ha terminado después de tomar la acción
    next_state, reward, done, _, _ = env.step(action)

    # El agente almacena la experiencia y aprende de ella
    agent.step(state, action, reward, next_state, done)

    # Actualiza el estado actual al siguiente estado
    state = next_state

    # Suma la recompensa obtenida en este paso al total del episodio
    score += reward

    # Si el episodio ha terminado (el entorno indica "done"), sale del bucle
    if done:
      break

  # Almacena la puntuación del episodio en una lista que guarda las últimas 100 puntuaciones
  scores_on_100_episodes.append(score)

  # Actualiza el valor de epsilon, disminuyéndolo para reducir gradualmente la exploración
  epsilon = max(epsilon_ending_value, epsilon_decay_value * epsilon)

  # Imprime el número del episodio y la puntuación promedio de los últimos 100 episodios
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")

  # Cada 100 episodios, imprime la puntuación promedio de los últimos 100 episodios
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))

  # Si la puntuación promedio en los últimos 100 episodios alcanza un umbral de 200.0 (considerado resuelto), guarda el modelo y termina el entrenamiento
  if np.mean(scores_on_100_episodes) >= 200.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
    torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
    break


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Episode 100	Average Score: -153.69
Episode 200	Average Score: -89.74
Episode 300	Average Score: -98.00
Episode 400	Average Score: -51.49
Episode 500	Average Score: -14.98
Episode 600	Average Score: -6.67
Episode 700	Average Score: 21.92
Episode 800	Average Score: 167.93
Episode 849	Average Score: 200.40
Environment solved in 749 episodes!	Average Score: 200.40


## Ver resultados

In [10]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder


# Generar el video
def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(agent, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()